Semantic search allows finding meaning/intent/emotion in queries, not just searching based on keywords.

In [80]:
!pip3 install -U langchain-community
!pip3 install faiss-cpu sentence-transformers
!pip3 install transformers langchain
!pip3 install -U langchain-huggingface


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


## Comparing Cosine Similarity Between Sentence Vectors
 
Convert sentences into Term Frequency-Inverse Document Frequency (TF-IDF) numerical vectors. Then calculate cosine similarity.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example sentences
sentence1 = "Hello darkness my old friend."
sentence2 = "Hello brightness my new enemy"

# Step 1: Convert sentences to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])

# Step 2: Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

# Output the similarity score
print(f"Cosine Similarity: {cosine_sim[0][0]}")

Cosine Similarity: 0.25233420143369617


## Load Documents from Corpus

In [38]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


datapath = "corpus"
loader = DirectoryLoader(
    datapath, 
    glob="*.txt",
    loader_cls=TextLoader
)

all_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(all_docs)

# Output the split documents
for doc in split_docs:
    print(doc)

page_content='IETN RIUOGvmrw jgouipq
g
3rg
regv
3rgv
re
bvr
r
g

g3r
gv3
rgv
rb

    Indented.' metadata={'source': 'corpus/test2.txt'}
page_content='This is a test paragraph 1.

This is a test paragraph 2.

Suck my balls.

Goodbye!' metadata={'source': 'corpus/test.txt'}


In [39]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document

## Create Embedding Function

In Langchain, embedding function has its own class and must be implemented.

In [43]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        # Load the Sentence Transformers model
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        # Generate embeddings for a list of texts (documents)
        return self.model.encode(texts, convert_to_tensor=False).tolist()

    def embed_query(self, text: str) -> list[float]:
        # Generate embedding for a single query
        return self.model.encode(text, convert_to_tensor=False).tolist()

## Create Embedding Model
Embedding model converts text into multi-dimensional numerical vectors (embeddings). The goal is to create a FAISS vector store which we can query for similar docs.

In [47]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

/var/folders/n5/frcrqltj3pzfngfbj79n6cm00000gp/T/ipykernel_17249/325135762.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Create FAISS Vector Store

In [63]:
faiss_store = FAISS.from_documents(all_docs, embedding_model)
retriever = faiss_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

## Query Embedding Model

In [64]:
query = "IETN RIUOGvmrw jgouipq"
results = retriever.invoke(query)

In [65]:
for result in results:
    print(result.page_content, result.metadata)


IETN RIUOGvmrw jgouipq
g
3rg
regv
3rgv
re
bvr
r
g

g3r
gv3
rgv
rb

    Indented. {'source': 'corpus/test2.txt'}


A chain is similar to a Haystack pipeline. They are nodes that process inputs and pass outputs to the next node in the chain.

A **retriever-based chain** will have a retriever node as a core component.

In [81]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Step 1: Load a Hugging Face model locally
model_name = "gpt2"  # Replace with your desired local model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 2: Create a text generation pipeline
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=256)

# Step 3: Wrap the pipeline with HuggingFacePipeline
local_llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use mps:0


In [86]:
from langchain.llms import OpenAI

# Step 4: Prepare documents
documents = [
    Document(page_content="Paris is the capital of France.", metadata={"source": "doc1"}),
    Document(page_content="Berlin is the capital of Germany.", metadata={"source": "doc2"}),
]

# Step 5: Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

# Step 6: Create a FAISS vector store
faiss_store = FAISS.from_documents(documents, embedding_model)

# Step 7: Initialize the retriever
retriever = faiss_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

retrieval_pipeline = RetrievalQA.from_chain_type(
    llm=local_llm,
    retriever=retriever,
    chain_type="stuff",  # Chain type can be adjusted for more complex tasks
)

In [87]:
query = "What is the capital of France?"
response = retrieval_pipeline.run(query)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/site-packages/transformers/pytorch_utils.py:335: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [88]:
response

'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nParis is the capital of France.\n\nQuestion: What is the capital of France?\nHelpful Answer: Paris is situated in France, and to know that it is a city, go to the capital, say, "Oh, it is a city, its capital. Paris is the capital of France."\n\nParis is an island, and its capital is Paris.\n\nTo use a different context, you first have to understand the meaning of "London", the capital of London.\n\nLondon is the capital of London, while Paris is the capital of Paris. London is the Capital of London, but not Paris.\n\nTo understand London, you have to understand that London is a city, and that it is a city.\n\nIt does not matter if the capital of France is Paris, London, or London, that is the same, at least there is no difference in the meaning of "London", "Paris", or "Paris".\n\nNow you need help.\n\nTry to understan